In [9]:
!pwd
!cd /vsphhome/xwx/Model/Geneformer/
!pwd

/vsphhome/xwx/Geneformer/examples
/vsphhome/xwx/Geneformer/examples


In [1]:
import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
from geneformer import InSilicoPerturber
from geneformer import InSilicoPerturberStats
from geneformer import EmbExtractor

### in silico perturbation in deletion mode to determine genes whose deletion in the dilated cardiomyopathy (dcm) state significantly shifts the embedding towards non-failing (nf) state

In [3]:
# imports
from collections import Counter
import datetime
import pickle
import subprocess
import seaborn as sns; sns.set()
from datasets import load_from_disk
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertForSequenceClassification
from transformers import Trainer
from transformers.training_args import TrainingArguments

from geneformer import DataCollatorForCellClassification
# load cell type dataset (includes all tissues)


In [4]:
current_date = datetime.datetime.now()
datestamp = f"{str(current_date.year)[-2:]}{current_date.month:02d}{current_date.day:02d}"

perturb_type="delete" #“overexpress”
pertur_stat_mode="goal_state_shift" #"aggregate_gene_shifts"#

output_prefix=f"depression_classificaton_checkpoint12278_ConMDD_{datestamp}_{perturb_type}_{pertur_stat_mode}"
pertur_stat_path="/vsphhome/xwx/Geneformer/perturbation_get_stats"
pertur_path=f"/vsphhome/xwx/Geneformer/perturbation_output/MDD_checkpoint12278/ConMDD_{perturb_type}"
# os.mkdir(pertur_path)
input_data_path="/vsphhome/xwx/Geneformer/token_data/MMD_snRNA.dataset/" #"/vsphhome/xwx/Geneformer/token_data/MMD_snRNA_2000.dataset/" #
model_path="/vsphhome/xwx/Geneformer/models/240329_geneformer_DepressionClassifier_L2048_B12_LR5e-05_LSlinear_WU500_E10_Oadamw_F3/checkpoint-12278"

In [5]:
cell_states_to_model={"state_key": "label", 
                      "start_state": "Control", 
                      "goal_state": "Suicide"
                      }


In [17]:
# first obtain start, goal, and alt embedding positions
# this function was changed to be separate from perturb_data
# to avoid repeating calcuations when parallelizing perturb_data
cell_states_to_model={"state_key": "label", 
                      "start_state": "Control", 
                      "goal_state": "Suicide"
                      }#"alt_states": ["hcm"]

embex = EmbExtractor(model_type="CellClassifier",
                     num_classes=2,
                     max_ncells=1000,
                     emb_layer=0,
                     summary_stat="exact_mean",
                     forward_batch_size=16,#256
                     nproc=16) 

state_embs_dict = embex.get_state_embs(cell_states_to_model,
                                       model_path, #"path/to/model",
                                       input_data_path, #"path/to/input_data",
                                       pertur_path, #"path/to/output_directory",
                                       output_prefix#"output_prefix"
                                       )

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

In [18]:
isp = InSilicoPerturber(perturb_type=perturb_type,
                        perturb_rank_shift=None,
                        genes_to_perturb="all",
                        combos=0,
                        anchor_gene=None,
                        model_type="CellClassifier",
                        num_classes=2,
                        emb_mode="cell",
                        cell_emb_style="mean_pool",
                        cell_states_to_model=cell_states_to_model,
                        state_embs_dict=state_embs_dict,
                        max_ncells=1000,
                        emb_layer=0,
                        forward_batch_size=50,
                        nproc=16)

In [19]:
# outputs intermediate files from in silico perturbation
isp.perturb_data(model_path, #"path/to/model",
                 input_data_path, #"path/to/input_data",
                 pertur_path,#"path/to/output_directory",
                 output_prefix #"output_prefix"
                 )



  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Map (num_proc=16):   0%|          | 0/2048 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/2048 [00:00<?, ? examples/s]

  0%|          | 0/41 [00:00<?, ?it/s]

...

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Map (num_proc=16):   0%|          | 0/1359 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/1359 [00:00<?, ? examples/s]

  0%|          | 0/28 [00:00<?, ?it/s]

In [6]:

ispstats = InSilicoPerturberStats(mode=pertur_stat_mode,
                                  genes_perturbed="all",
                                  combos=0,
                                  anchor_gene=None,
                                  cell_states_to_model=cell_states_to_model)

In [7]:
# extracts data from intermediate files and processes stats to output in final .csv
ispstats.get_stats(pertur_path, #"path/to/input_data",
                   None,
                   pertur_stat_path, #"path/to/output_directory",
                   output_prefix
                   )

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/15890 [00:00<?, ?it/s]

  0%|          | 0/15890 [00:00<?, ?it/s]

/vsphhome/xwx/anaconda3/envs/gene3/lib/python3.11/site-packages/geneformer/in_silico_perturber_stats.py:402: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cos_sims_full_df = pd.concat([cos_sims_full_df, cos_sims_df_i])
